# Gerar 10 folds para validação cruzada do conjunto de dados Cohebert manual

Divide os dados do Cohebert Manual em 10 folds.

Cada fold possui 90% para dados de treinamento e 10% para dados de teste.

Os dados de teste não se repetem em cada fold.

O procedimento exclui os documentos que geram mais de 512 tokens.

Utiliza os arquivos para gerar K folds:
- `original.zip`
- `originalpos.zip`
- `perturbado_pX_kY.zip`
- `perturbadopos_pX_kY.zip`

Nos nomes dos arquivos `perturbado_pX_kY.zip`,`perturbadopos_pX_kY.zip`, X é o número de documentos perturbados e Y o valor de top K predições. 

Cria o arquivo `COHEBERT_KFOLD_10_PX_KY.zip`, X é o número de documentos perturbados e Y o valor de top K predições.

----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do notebook.

## 1.1 Tempo inicial de processamento

In [ ]:
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

In [ ]:
print("  Tempo de início de processamento:  {:} (h:mm:ss)".format(inicio_processamento))

  Tempo de início de processamento:  1645786325.6248918 (h:mm:ss)


## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """    
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
import time
import datetime

def formataTempo(tempo):
    '''
    Pega a tempo em segundos e retorna uma string hh:mm:ss
    '''
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Calcula a média de uma lista tempo string no formato hh:mm:ss.

In [ ]:
# Import das bibliotecas.
from cmath import rect, phase
from math import radians, degrees
  
def mediaAngulo(deg):
    return degrees(phase(sum(rect(1, radians(d)) for d in deg)/len(deg)))
 
def mediaTempo(tempos):
    '''
    Calcula a média de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Verifica se deu algum dia
    dia = 24 * 60 * 60
    # Converte para angulos
    para_angulos = [s * 360. / dia for s in segundos]
    # Calcula a média dos angulos
    media_como_angulo = mediaAngulo(para_angulos)
    media_segundos = media_como_angulo * dia / 360.
    if media_segundos < 0:
        media_segundos += dia
    # Recupera as horas e os minutos  
    h, m = divmod(media_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)    
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Calcula a soma de uma lista de tempo string no formato hh:mm:ss

In [ ]:
def somaTempo(tempos):
    '''
    Calcula a soma de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Soma os segundos
    soma_segundos = sum([s * 1. for s in segundos])
    # Recupera as horas e os minutos   
    h, m = divmod(soma_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)    
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )

Classe(ModeloArgumentosClassificacao) de definição dos parâmetros do modelo para classificação

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosClassificacao:
    '''
    Classe(ModeloArgumentosClassificacao) de definição dos parâmetros do modelo BERT para a classificação de coerência.
    '''
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )
    num_labels: int = field(
        default=2,
        metadata={"help": "número de rótulos a serem classificados."},
    )
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    optimizer: str = field(
        default="AdamW",
        metadata={"help": "otimizador do modelo."},
    )
    use_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o uso do wandb."},
    )
    salvar_modelo_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do modelo no wandb."},
    )
    salvar_modelo : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento do modelo."},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do resultado da avaliação."},
    )     
    salvar_classificacao : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento da classificação."},
    )
    usar_mcl_ajustado: bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )   
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos perturbados comparados com o seu original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de previsões de palavras recuperadas mais próximas da máscara."},
    )
    fold: int = field(
        default="1",
        metadata={"help": "fold a ser avaliado."},
    )    

Biblioteca de limpeza de tela

In [ ]:
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Biblioteca de logging
import logging

# Formato da mensagem
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

2022-02-25 10:52:08,981 : INFO : Encontrei GPU em: /device:GPU:0


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.
    
    Retorno:
    `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
device = getDeviceGPU()

2022-02-25 10:52:09,646 : INFO : Existem 1 GPU(s) disponíveis.
2022-02-25 10:52:14,004 : INFO : Iremos usar a GPU: Tesla P100-PCIE-16GB.


Conecta o modelo ao device

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.       
        `device` - Um device de GPU.     
    
      Retorno:
        `model` - Um objeto model BERT conectado a GPU.     
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():    
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()
        
    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [ ]:
# Import das bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

2022-02-25 10:52:14,054 : INFO : Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

2022-02-25 10:52:14,058 : INFO : Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
2022-02-25 10:52:14,062 : INFO : e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.



In [ ]:
# Monta o Google Drive para esta instância de notebook.
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

Login via linha de comando

In [ ]:
!wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.8 Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
!pip install -U transformers==4.5.1

# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros a serem avaliados
# Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 1

# Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 1

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosClassificacao(     
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",   
    do_lower_case = False,   # default True
    num_labels = 2,
    output_attentions = False,    # default False
    output_hidden_states = False, # default False 
    optimizer = 'AdamW',
    use_wandb = False, # Ativa a gravação de logs no wandb
    salvar_modelo_wandb = False, # Ativa o salvamento do MCL no wandb
    salvar_modelo = False, # Ativa o salvamento do MCL no googledrive
    salvar_avaliacao = False, # Salva o resultado classificações
    salvar_classificacao = False, # Salva o resultado da avaliação das classificações
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path. 
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.    
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara. 
    fold = 10 # fold = 10 # Quantidade de folds a serem gerados.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHEBERT_MANUAL"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):  
      # Cria o diretório
      os.makedirs(dirbase)    
      logging.info("Diretório criado: {}.".format(dirbase))
  else:    
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

2022-02-25 14:56:28,759 : INFO : Diretório criado: /content/COHEBERT_MANUAL.


Diretório para conter as os resultados das classificações

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "/validacao_classificacao_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacao()

2022-02-25 14:56:29,411 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL//validacao_classificacao_palavra.


Diretório para conter os arquivos da avaliação kfold

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacaoKfold():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_classificacao_palavra/kfold"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacaoKfold()

2022-02-25 14:56:29,767 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_classificacao_palavra/kfold.


# 3 BERT

## 3.1 Modelo Pré-treinado BERT

### Funções Auxiliares

In [ ]:
def getNomeModeloBERT(model_args):
    '''    
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"
    
    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"
        
    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"
            
    return MODELO_BERT

In [ ]:
def getTamanhoBERT(model_args):
    '''    
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''
    
    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"
    
    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"
        
    return TAMANHO_BERT  

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 
    
    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO
        
        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"
        
        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo!")
            # Apaga o diretório e os arquivos existentes                     
            shutil.rmtree(DIRETORIO_MODELO)
        
        # Realiza o download do arquivo do modelo        
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.                
        arquivo_zip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivo_zip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.        
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)
        
        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta!".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada!")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """ 
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO) 
   
    logging.info("Modelo BERT ajustado copiado!")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """ 
    Verifica de onde utilizar o modelo.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.
    
    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 

    DIRETORIO_MODELO = None
    
    if model_args.usar_mcl_ajustado == True:        
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()
        
        logging.info("Usando modelo BERT ajustado.")
        
    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")        
        
    return DIRETORIO_MODELO

## 3.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).



In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.       
    
      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None
    
    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 3.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

### Função carrega modelo BERT medida




In [ ]:
# Import das bibliotecas.
from transformers import BertModel # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.   
    
      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:        
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertModel.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)
        
    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertModel.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega modelo BERT classificação



In [ ]:
# Import das bibliotecas.
from transformers import BertForSequenceClassification # Importando as bibliotecas do Modelo BERT.

def carregaModeloClassifica(DIRETORIO_MODELO, model_args):
    ''' 
    Carrega o modelo e retorna o modelo.
    
    Parâmetros:
    `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
    `model_args` - Objeto com os argumentos do modelo.
    
    Retorno:
    `model` - Um objeto do modelo BERT carregado.
    ''' 

    # Variável para setar o arquivo.
    URL_MODELO = None

    if 'http' in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para classificação.".format(DIRETORIO_MODELO))

        model = BertForSequenceClassification.from_pretrained(DIRETORIO_MODELO, 
                                                              num_labels=model_args.num_labels,
                                                              output_attentions=model_args.output_attentions,
                                                              output_hidden_states=model_args.output_hidden_states)
            
    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para classificação.".format(model_args.pretrained_model_name_or_path))

        model = BertForSequenceClassification.from_pretrained(model_args.pretrained_model_name_or_path,
                                                              num_labels=model_args.num_labels,
                                                              output_attentions=model_args.output_attentions,
                                                              output_hidden_states=model_args.output_hidden_states)
    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """ 
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:    
        `model` - Um objeto do modelo BERT carregado.       
        `tokenizer` - Um objeto tokenizador BERT carregado.       
    """
            
    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)
    
    # Variável para conter o modelo
    model = None
    
    # Verifica o tipo do modelo em model_args    
    if type(model_args) == ModeloArgumentosMedida:
        # Carrega o modelo para cálculo da medida
        model = carregaModeloMedida(DIRETORIO_MODELO, model_args)
        
    else:
        # Carrega o modelo para classificação
        model = carregaModeloClassifica(DIRETORIO_MODELO, model_args)
        
        # Recupera o dispotivo da GPU 
        device = getDeviceGPU()
    
        # Conecta o modelo a GPU
        model = conectaGPU(model, device)
                       
    # Carrega o tokenizador. 
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)
    
    return model, tokenizer

### Carrega o BERT

In [ ]:
# Carrega o modelo e tokenizador do BERT
model, tokenizer = carregaBERT(model_args)

2022-02-25 10:52:29,686 : INFO : Diretório Cohebert criado: COHEBERT_MANUAL
2022-02-25 10:52:30,137 : INFO : Download do arquivo: bert-large-portuguese-cased_pytorch_checkpoint.zip.


  0%|          | 0.00/1.24G [00:00<?, ?B/s]

2022-02-25 10:53:20,549 : INFO : Download do arquivo: COHEBERT_MANUAL/modeloBERT/vocab.txt.


  0%|          | 0.00/210k [00:00<?, ?B/s]

2022-02-25 10:53:20,746 : INFO : Diretório COHEBERT_MANUAL/modeloBERT do modelo BERT pronta!
2022-02-25 10:53:20,880 : INFO : Usando modelo BERT pré-treinado.
2022-02-25 10:53:20,881 : INFO : Carregando o modelo BERT do diretório COHEBERT_MANUAL/modeloBERT para classificação.
Some weights of the model checkpoint at COHEBERT_MANUAL/modeloBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertFor

### Recupera detalhes do BERT

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = getNomeModeloBERT(model_args)

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = getTamanhoBERT(model_args)

# 4 Divisão dos Folds

## 4.1 Carregamento dos arquivos de dados originais

### 4.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

### 4.1.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

2022-02-25 10:53:29,250 : INFO : Diretório já existe: /content/COHEBERT_MANUAL.


### 4.1.3 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"
 
  logging.info("Terminei a cópia.")

2022-02-25 10:53:31,020 : INFO : Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

2022-02-25 10:53:31,492 : INFO : Terminei a descompactação.


### 4.1.4 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-02-25 10:53:31,522 : INFO : TERMINADO ORIGINAIS: 40.
2022-02-25 10:53:31,523 : INFO : TERMINADO ORIGINAIS POS: 40.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento,classe
17,18,['O que é uma fila e como empilhar seu element...,O que é uma fila e como empilhar seu elemento?,0
2,3,['Como desenfileirar elementos em uma fila?'],Como desenfileirar elementos em uma fila?,1
33,34,['Como são implementadas as operações de empil...,Como são implementadas as operações de empilha...,0
37,38,['Em uma pilha a operação de enfileirar ocorre...,Em uma pilha a operação de enfileirar ocorre e...,0
30,31,['O que é uma fila e como enfileirar e desenfi...,O que é uma fila e como enfileirar e desenfile...,1


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
30,31,"[[['O', 'que', 'é', 'uma', 'fila', 'e', 'como'..."
20,21,"[[['O', 'que', 'é', 'uma', 'fila', 'e', 'como'..."
3,4,"[[['Como', 'desenfileirar', 'elementos', 'em',..."
26,27,"[[['O', 'que', 'é', 'uma', 'pilha', 'e', 'como..."
16,17,"[[['O', 'que', 'é', 'uma', 'pilha', 'e', 'como..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-02-25 10:53:31,886 : INFO : TERMINADO CORREÇÃO ORIGINAIS: 40.
2022-02-25 10:53:31,894 : INFO : TERMINADO CORREÇÃO ORIGINAIS POS: 40.


### 4.1.5 Renomeia o dataframe dos dados

In [ ]:
dfdados = lista_documentos_originais

Apaga as colunas desnecessárias

In [ ]:
dfdados = dfdados.drop(columns=["sentencas"])

In [ ]:
dfdados.sample(5)

,id,documento,classe
29,30,O que é uma fila e como empilhar e desempilhar...,0
12,13,Como desempilhar elementos em uma pilha?,1
23,24,O que é uma fila e como desempilhar um element...,0
30,31,O que é uma fila e como enfileirar e desenfile...,1
0,1,Como enfileirar elementos em uma fila?,1


## 4.2 Preparação dos dados

### Descartando os documentos muito grandes

Descarta os documentos que geram mais de 512 tokens com o tokenizador do BERT.

### Função descarte documentos muito grandes

In [ ]:
def descarteDocumentosGrandes(tamanho_maximo_token, dfdados):
  
  # Define o tamanho máximo para os tokens.
  tamanho_maximo = tamanho_maximo_token

  # Tokenize a codifica as setenças para o BERT.     
  dfdados["input_ids"] = dfdados["documento"].apply(lambda tokens: tokenizer.encode(tokens, add_special_tokens=True))
        
  dfdados = dfdados[dfdados["input_ids"].apply(len)<tamanho_maximo]

  logging.info("Tamanho do conjunto de dados: {}.".format(len(dfdados)))

  # Remove as colunas desnecessárias.
  dfdados = dfdados.drop(columns=["input_ids"])

  # Informações do DataFrame
  print(dfdados.info())

  return dfdados

### Descartando documentos muito grandes

In [ ]:
dfdados = descarteDocumentosGrandes(model_args.max_seq_len, dfdados)

2022-02-25 10:53:31,980 : INFO : Tamanho do conjunto de dados: 40.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         40 non-null     int64 
 1   documento  40 non-null     object
 2   classe     40 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB
None


### Procurando a maior quantidade de tokens no conjunto de dados

As células abaixo executará uma passagem de tokenização do conjunto de dados de treino, avaliação e teste para medir o comprimento máximo das documentos.

In [ ]:
maior_tamanho = 0

# Para cada documento no conjunto de treino.
for index, linha in dfdados.iterrows():        

    # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`.
    input_ids = tokenizer.encode(linha["documento"], add_special_tokens=True)

    # Atualiza o tamanho máximo de documento.
    maior_tamanho = max(maior_tamanho, len(input_ids))

logging.info("Tamanho máximo de documento no conjunto de dados: {}.".format(maior_tamanho))

2022-02-25 10:53:32,026 : INFO : Tamanho máximo de documento no conjunto de dados: 24.


## 4.3 Gerando os pares originais e permutados

Atributos da lista:
0. "id_original" - Nome do arquivo original
1. "documento_original" - Documento original
2. "classe_original" - Classe do documento original (1)
3. "id_permutado" - Nome do arquivo permutado
4. "documento_permutado" - Documento permutado
5. "classe_permutado" - Classe do documento permutado (0)

In [ ]:
lista  = dfdados.values.tolist()

# Lista dos documentos originais e permutados 
lista_documentos = []

# Para cada documento no conjunto de treino.
pos = 0
while pos < len(lista):     

  lista_documentos.append([ lista[pos][0], 
                           lista[pos][1], 
                           lista[pos][2], 
                           lista[pos+1][0], 
                           lista[pos+1][1], 
                           lista[pos+1][2]])

  pos = pos + 2

### Transforma a lista em um dataframe

Atributos do dataframe:
0. "id_original" - Nome do arquivo original
1. "documento_original" - Documento original
2. "classe_original" - Classe do documento original (1)
3. "id_permutado" - Nome do arquivo permutado
4. "documento_permutado" - Documento permutado
5. "classe_permutado" - Classe do documento permutado (0)

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Converte a lista em um dataframe.
dfdados = pd.DataFrame.from_records(lista_documentos, columns=["id_original",
                                                               "documento_original",
                                                               "classe_original",
                                                               "id_pPerturbado",
                                                               "documento_perturbado",
                                                               "classe_perturbado"])

# Número de linhas carregadas do arquivo.
logging.info("Total de registros              : {}.".format(len(dfdados)))

2022-02-25 10:53:32,050 : INFO : Total de registros              : 20.


Usaremos os pandas para analisar o conjunto de dados e examinar algumas de suas propriedades e pontos de dados.

In [ ]:
dfdados.sample(5)

,idOriginal,documentoOriginal,classeOriginal,idPerturbado,documentoPerturbado,classePerturbado
6,13,Como desempilhar elementos em uma pilha?,1,14,Como desempilhar elementos em uma fila?,0
8,17,O que é uma pilha e como empilhar seu elemento?,1,18,O que é uma fila e como empilhar seu elemento?,0
13,27,O que é uma pilha e como empilhar um elemento ...,1,28,O que é uma fila e como empilhar um elemento n...,0
15,31,O que é uma fila e como enfileirar e desenfile...,1,32,O que é uma pilha e como enfileirar e desenfil...,0
0,1,Como enfileirar elementos em uma fila?,1,2,Como enfileirar elementos em uma pilha?,0


## 4.3 Gerar divisão em k folds

### 4.3.1 Especifica os nomes dos arquivos de dados

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_FOLD_COMPACTADO = DIRETORIO_COHEBERT + "_KFOLD_10" + "_P" +  str(model_args.documentos_perturbados) + "_K" + str(model_args.top_k_predicao) + ".zip"

### 4.3.2 Realizando a divisão do conjunto de dados para treinamento e validação através do k folds

In [ ]:
# Import das bibliotecas.
import numpy as np
from sklearn.model_selection import KFold

# Cria o diretório para receber os folds
# Diretório a ser criado
dirbase = "/content/validacao_kfold"

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

# Parâmetros.
QTDE_FOLDS = model_args.fold

# Define o prefixo do nome dos arquivos dos folds
PREFIXO_NOME_ARQUIVO_TREINO = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Train_f"
PREFIXO_NOME_ARQUIVO_TESTE = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Test_f"

# Registra o tempo inícial.
t0 = time.time()

# Preparação do conjunto de dados.
X =  np.array(dfdados)

# Divisão em k folds(n_splits).
# shuffle, embaralha cada amostra.
# Quando shuffle igual True, random_state afeta a ordem dos índices,
# que controla a aleatoriedade de cada dobra.
kf = KFold(n_splits=QTDE_FOLDS,  random_state=True, shuffle=True)
print(kf)

TESTE = 1

# Percorre os indices do conjunto de dados.
for train_index, test_index in kf.split(X):
  logging.info("Executando divisão do fold: {}, Total: {}.".format(TESTE, len(train_index)+len(test_index)))
  logging.info("Treino: {}, Teste: {}.".format(len(train_index), len(test_index)))

  #print("Índices de treino:", len(train_index), " - ", train_index[0], " - ", train_index[len(train_index)-1])  
  #print(train_index)
  #print("Índices de teste :", len(test_index), "  - ", test_index[0], " - ", test_index[len(test_index)-1])
  #print(test_index)

  # Recupera os dados das documentos para treino e teste.
  documentos_train = X[train_index]
  documentos_test  = X[test_index]  

  # Organiza dados de treino.
  documentos_train_organizada = []

  # Adiciona a documento incoerente logo após a coerente para treino.
  for linha in documentos_train:     
    # Documento original
    # linha[0] = "idOriginal" linha[1] = "documentoOriginal" linha[2] = "classeOriginal"
    documentos_train_organizada.append((linha[0],linha[1],linha[2]))  
    # Documento permutado  
    # linha[3] = "idPermutado" linha[4] = "documentoPermutado" linha[5] = "classePermutado"
    documentos_train_organizada.append((linha[3],linha[4],linha[5]))
    
  # Cria um dataframe com os dados de treinamento.
  pddata_tuples_train = pd.DataFrame(documentos_train_organizada, columns=["id","documento","classe"])
    
  # Salva o arquivo de treino do fold.
  pddata_tuples_train.to_csv(PREFIXO_NOME_ARQUIVO_TREINO + str(TESTE)+".csv", index = False, sep=';')

  # Organiza dados de teste.
  documentos_test_organizada = []

  # Adiciona a documento incoerente logo após a coerente para teste.
  for linha in documentos_test:    
    # 1 Para coerente.
    # linha[0] = "id_original" linha[1] = "documento_original" linha[2] = "classe_original"
    documentos_test_organizada.append((linha[0],linha[1],linha[2]))
    # 0 para uma permutação como incoerente.
    # linha[3] = "id_permutado" linha[4] = "documento_permutado" linha[5] = "classe_permutado"
    documentos_test_organizada.append((linha[3],linha[4],linha[5]))
    
  # Cria um dataframe com os dados de teste.
  pddata_tuples_test = pd.DataFrame(documentos_test_organizada, columns=["id","documento","classe"])  
  
  # Salva o arquivo de teste do fold.
  pddata_tuples_test.to_csv(PREFIXO_NOME_ARQUIVO_TESTE+str(TESTE)+".csv", index = False, sep=';')

  # Avança o contador de testes.
  TESTE = TESTE + 1

# Medida de quanto tempo levou a execução da validação.
teste_time = formataTempo(time.time() - t0)

logging.info("  Tempo gasto: {:}.".format(teste_time))

2022-02-25 10:53:32,354 : INFO : Diretório criado: /content/validacao_kfold.
2022-02-25 10:53:32,360 : INFO : Executando divisão do fold: 1, Total: 20.
2022-02-25 10:53:32,361 : INFO : Treino: 18, Teste: 2.
2022-02-25 10:53:32,370 : INFO : Executando divisão do fold: 2, Total: 20.
2022-02-25 10:53:32,371 : INFO : Treino: 18, Teste: 2.
2022-02-25 10:53:32,379 : INFO : Executando divisão do fold: 3, Total: 20.
2022-02-25 10:53:32,380 : INFO : Treino: 18, Teste: 2.
2022-02-25 10:53:32,390 : INFO : Executando divisão do fold: 4, Total: 20.
2022-02-25 10:53:32,392 : INFO : Treino: 18, Teste: 2.
2022-02-25 10:53:32,398 : INFO : Executando divisão do fold: 5, Total: 20.
2022-02-25 10:53:32,400 : INFO : Treino: 18, Teste: 2.
2022-02-25 10:53:32,406 : INFO : Executando divisão do fold: 6, Total: 20.
2022-02-25 10:53:32,407 : INFO : Treino: 18, Teste: 2.
2022-02-25 10:53:32,414 : INFO : Executando divisão do fold: 7, Total: 20.
2022-02-25 10:53:32,415 : INFO : Treino: 18, Teste: 2.
2022-02-25 10

KFold(n_splits=10, random_state=True, shuffle=True)


### 4.3.2 Analisando os dados gerados na divisão

In [ ]:
# Lista para armazenar os dados
lista_dados_train_folds = []
lista_dados_test_folds = []

# Define o prefixo do nome dos arquivos dos folds
PREFIXO_NOME_ARQUIVO_TREINO = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Train_f"
PREFIXO_NOME_ARQUIVO_TESTE = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Test_f"

for x in range(QTDE_FOLDS):
  
  dadostrain = pd.read_csv(PREFIXO_NOME_ARQUIVO_TREINO+str(x+1)+".csv", sep=';')
  logging.info("Dados treino do fold {}: {}.".format(x+1,len(dadostrain)))

  dadostest = pd.read_csv(PREFIXO_NOME_ARQUIVO_TESTE+str(x+1)+".csv", sep=';')
  logging.info("Dados teste do fold {}: {}.".format(x+1,len(dadostest)))

  lista_dados_train_folds.append([x,dadostrain.classe.sum(), len(dadostrain.classe)-dadostrain.classe.sum()])
  lista_dados_test_folds.append([x,dadostest.classe.sum(), len(dadostest.classe)-dadostest.classe.sum()])

  # Pega as listas de documentos e seus rótulos.
  documentos = dadostrain.documento.values
  maior_tamanho_documento_treino = 0
  # Para cada documento no conjunto de treino.
  for documento in documentos:
      # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`.
      input_ids = tokenizer.encode(documento, add_special_tokens=True)
      # Atualiza o tamanho máximo de documento.
      maior_tamanho_documento_treino = max(maior_tamanho_documento_treino, len(input_ids))

  logging.info("Máximo de tokens no conjunto de dados de treino: {}.".format(maior_tamanho_documento_treino))

  # Pega as listas de documentos e seus rótulos.
  documentos = dadostest.documento.values
  maior_tamanho_documento_teste = 0    
  # Para cada documento no conjunto de treino.  
  for documento in documentos:
      # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`.
      input_ids = tokenizer.encode(documento, add_special_tokens=True)
      # Atualiza o tamanho máximo de documento.
      maior_tamanho_documento_teste = max(maior_tamanho_documento_teste, len(input_ids))

  logging.info("Máximo de token no conjunto de dados de teste: {}.".format(maior_tamanho_documento_teste))
                               
  logging.info("Fold {} Treino positivos: {} de {} ({:.2f}%)".format(x+1, 
                                                             dadostrain.classe.sum(), 
                                                             len(dadostrain.classe), 
                                                             (dadostrain.classe.sum() / len(dadostrain.classe) * 100.0)
                                                             ))
  
  logging.info("Fold {} Treino negativos: {} de {} ({:.2f}%)".format(x+1, 
                                                             len(dadostrain.classe)-dadostrain.classe.sum(), 
                                                             len(dadostrain.classe), 
                                                             ((len(dadostrain.classe)-dadostrain.classe.sum()) / len(dadostrain.classe) * 100.0)))

  logging.info("Fold {} Teste positivos: {} de {} ({:.2f}%)".format(x+1, 
                                                             dadostest.classe.sum(), 
                                                             len(dadostest.classe), 
                                                             (dadostest.classe.sum() / len(dadostest.classe) * 100.0)))
  logging.info("Fold {} Teste negativos: {} de {} ({:.2f}%)".format(x+1, 
                                                             len(dadostest.classe)-dadostest.classe.sum(), 
                                                             len(dadostest.classe), 
                                                             ((len(dadostest.classe)-dadostest.classe.sum()) / len(dadostest.classe) * 100.0)))                               
  logging.info("")


2022-02-25 10:53:32,503 : INFO : Dados treino do fold 1: 36.
2022-02-25 10:53:32,507 : INFO : Dados teste do fold 1: 4.
2022-02-25 10:53:32,522 : INFO : Máximo de tokens no conjunto de dados de treino: 24.
2022-02-25 10:53:32,527 : INFO : Máximo de token no conjunto de dados de teste: 23.
2022-02-25 10:53:32,528 : INFO : Fold 1 Treino positivos: 18 de 36 (50.00%)
2022-02-25 10:53:32,531 : INFO : Fold 1 Treino negativos: 18 de 36 (50.00%)
2022-02-25 10:53:32,533 : INFO : Fold 1 Teste positivos: 2 de 4 (50.00%)
2022-02-25 10:53:32,536 : INFO : Fold 1 Teste negativos: 2 de 4 (50.00%)
2022-02-25 10:53:32,539 : INFO : 
2022-02-25 10:53:32,543 : INFO : Dados treino do fold 2: 36.
2022-02-25 10:53:32,547 : INFO : Dados teste do fold 2: 4.
2022-02-25 10:53:32,560 : INFO : Máximo de tokens no conjunto de dados de treino: 24.
2022-02-25 10:53:32,570 : INFO : Máximo de token no conjunto de dados de teste: 18.
2022-02-25 10:53:32,571 : INFO : Fold 2 Treino positivos: 18 de 36 (50.00%)
2022-02-25 1

### 4.3.3 Compacta os arquivos dos folds

Compacta os arquivos em dos folds

A opção `-jr` do zip compacta os arquivos ignorando a estrutura de diretório.

In [ ]:
!zip -jr 'validacao_kfold/$NOME_ARQUIVO_FOLD_COMPACTADO' 'validacao_kfold/$DIRETORIO_COHEBERT'_*.csv

logging.info("Terminei a compactação.")

  adding: COHEBERT_MANUAL_Test_f10.csv (deflated 55%)
  adding: COHEBERT_MANUAL_Test_f1.csv (deflated 63%)
  adding: COHEBERT_MANUAL_Test_f2.csv (deflated 50%)
  adding: COHEBERT_MANUAL_Test_f3.csv (deflated 54%)
  adding: COHEBERT_MANUAL_Test_f4.csv (deflated 58%)
  adding: COHEBERT_MANUAL_Test_f5.csv (deflated 54%)
  adding: COHEBERT_MANUAL_Test_f6.csv (deflated 49%)
  adding: COHEBERT_MANUAL_Test_f7.csv (deflated 60%)
  adding: COHEBERT_MANUAL_Test_f8.csv (deflated 60%)
  adding: COHEBERT_MANUAL_Test_f9.csv (deflated 54%)
  adding: COHEBERT_MANUAL_Train_f10.csv (deflated 82%)
  adding: COHEBERT_MANUAL_Train_f1.csv (deflated 82%)
  adding: COHEBERT_MANUAL_Train_f2.csv (deflated 83%)
  adding: COHEBERT_MANUAL_Train_f3.csv (deflated 83%)
  adding: COHEBERT_MANUAL_Train_f4.csv (deflated 82%)
  adding: COHEBERT_MANUAL_Train_f5.csv (deflated 83%)
  adding: COHEBERT_MANUAL_Train_f6.csv (deflated 83%)
  adding: COHEBERT_MANUAL_Train_f7.csv (deflated 84%)
  adding: COHEBERT_MANUAL_Train_f8.c

2022-02-25 10:53:33,134 : INFO : Terminei a compactação.


Copia o arquivo compactado para o google drive

In [ ]:
!cp 'validacao_kfold/$NOME_ARQUIVO_FOLD_COMPACTADO' '/content/drive/My Drive/Colab Notebooks/Data/'$DIRETORIO_COHEBERT'/validacao_classificacao_palavra/kfold'

logging.info("Terminei a cópia.")

2022-02-25 10:53:33,573 : INFO : Terminei a cópia.


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:01:28 (h:mm:ss)
